In [ ]:
"""Code to generate a TED Talk"""

# Author: Sameera Lanka
# Website: www.sameera-lanka.com
# Copyright © TED Conferences, LLC

from trainTED import network, ip_dim, hi_dim, num_layers, char2oh, vocab 
import torch
from torch.autograd import Variable
from sys import stdout
    
def encode(letter):
    """One-hot encoding of letter"""
    return char2oh[letter]

def decode(tensor):
    """Letter from one-hot encoding"""
    index = tensor.view(-1).max(0)[1]
    return vocab[index.data[0]]

prompt = "The next big invention " #feed in a input prompt
speech_len = 1000

In [ ]:
if __name__=="__main__":
    
    speaker = network(ip_dim, hi_dim, num_layers).cuda()
    speaker.load_state_dict(torch.load('./winner.pth'))
    speaker.batch_size = 1
    speaker.reset()
    
    temperature = 0.55  
    
    for letter in prompt:
        ip = Variable(encode(letter).view(1,1,-1), volatile=True).cuda() 
        output = decode(speaker(ip))
        stdout.write(letter)
    
    for counter in range(speech_len):
        ip = Variable(encode(output).view(1,1,-1), volatile=True).cuda()
        output_dist = speaker(ip).data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        output = vocab[top_i]
        stdout.write(output)
        
    